In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

sns.set_style('darkgrid')
sns.set(font_scale=1.6)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
df_market = pd.read_csv('../analysis/data/GOOG_market.csv')

In [4]:
df_market.date = pd.to_datetime(df_market['date'])

Data Generator

In [5]:
def generator(data, lookback, min_index, max_index,
              shuffle=False, batch_size=128, step=1):
    if max_index is None:
        max_index = data.shape[0] - 1
        
    i = min_index + lookback
    while True:
        # Select the rows that will be used for the batch
        if shuffle:
            rows = np.random.randint(min_index + lookback, size=batch_size)
        else:
            rows = np.arange(i, min(i + batch_size, max_index))
            i += rows.shape[0]
            if i + batch_size >= max_index:
                i = min_index + lookback
                
        samples = np.zeros((len(rows),
                   lookback // step,
                   data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j]][1]
            
        yield samples, targets

In [6]:
from keras.callbacks import Callback

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

Using TensorFlow backend.


AttributeError: 'module' object has no attribute 'SSL_ST_INIT'